In [ ]:
# Setup Chrome WebDriver with headless option
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Comment out for debugging
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    service = Service('/path/to/chromedriver')  # Update this path
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver


In [ ]:
# Extract facility data using BeautifulSoup
def extract_data(soup):
    results = []
    listings = soup.find_all("div", class_="location-info")

    for item in listings:
        business_name = item.find("h3")
        business_name = business_name.get_text(strip=True) if business_name else ""

        update_tag = item.find("div", class_="updated-date")
        last_update_date = update_tag.get_text(strip=True) if update_tag else ""

        address = item.find("div", class_="address")
        street_address = address.get_text(" ", strip=True) if address else ""

        materials_section = item.find("div", class_="materials-accepted")
        materials = materials_section.find_all("li") if materials_section else []
        materials_accepted = ", ".join([m.get_text(strip=True) for m in materials])

        results.append({
            "business_name": business_name,
            "last_update_date": last_update_date,
            "street_address": street_address,
            "materials_accepted": materials_accepted
        })
    return results


In [ ]:
# Load page and scrape data
def scrape_facilities(search_material="glass", search_zip="90210"):
    url = f"https://search.earth911.com/?what={search_material}&where={search_zip}"
    driver = setup_driver()
    driver.get(url)
    time.sleep(5)  # wait for JS to render content

    soup = BeautifulSoup(driver.page_source, "html.parser")
    data = extract_data(soup)

    driver.quit()
    return data


In [ ]:
# Save data to CSV
def save_to_csv(data, filename="recycling_facilities.csv"):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"Saved {len(df)} entries to {filename}")


In [ ]:
# Run everything
if __name__ == "__main__":
    facilities = scrape_facilities(search_material="plastic", search_zip="10001")
    save_to_csv(facilities)
